<a href="https://colab.research.google.com/github/kazirafi17/DataScrapping/blob/main/Lugue1_last10years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import time

In [2]:
import pandas as pd
import requests

# Step 1: Fetch fixture data
url_fixtures = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
url_statistics = "https://api-football-v1.p.rapidapi.com/v3/fixtures/statistics"

headers = {
    "X-RapidAPI-Key": "b6e89817d6msh36107de73277139p116779jsne307fb015e33",
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

# Define league and season
league_id = "61"  # for Ligue 1
seasons = ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015']

# Fetch the fixtures
querystring_fixtures = {"league": league_id, "season": seasons}
response_fixtures = requests.get(url_fixtures, headers=headers, params=querystring_fixtures)

# Check if the request was successful
if response_fixtures.status_code == 200:
    data_fixtures = response_fixtures.json()
    fixtures = data_fixtures['response']

    # Step 2: Loop through each fixture to get the full statistics
    all_stats = []
    for fixture in fixtures:
        fixture_id = fixture['fixture']['id']  # Get the fixture ID
        fixture_date = fixture['fixture']['date']  # Get the fixture date

        # Fetch the statistics for this fixture
        querystring_stats = {"fixture": fixture_id}
        response_stats = requests.get(url_statistics, headers=headers, params=querystring_stats)

        if response_stats.status_code == 200:
            stats_data = response_stats.json()

            # Extract relevant stats for both teams
            if 'response' in stats_data and stats_data['response']:
                for team_stats in stats_data['response']:
                    team_name = team_stats['team']['name']
                    team_id = team_stats['team']['id']

                    # Create a dictionary to store all statistics for this team in this fixture
                    stats_dict = {
                        "fixture_id": fixture_id,
                        "fixture_date": fixture_date,
                        "team_name": team_name,
                        "team_id": team_id
                    }

                    # Loop through the available statistics and store them in the dictionary
                    for stat in team_stats['statistics']:
                        stat_name = stat['type']
                        stat_value = stat['value']
                        stats_dict[stat_name] = stat_value

                    # Add the statistics to the list
                    all_stats.append(stats_dict)
        else:
            print(f"Error fetching stats for fixture {fixture_id}: {response_stats.status_code}")
else:
    print(f"Error fetching fixtures: {response_fixtures.status_code}")

# Step 3: Convert the statistics list to a DataFrame
df_stats = pd.DataFrame(all_stats)

# Save to CSV to retain all stats for each team per fixture
df_stats.to_csv('Ligue1_full_stats_1.csv', index=False)
print("Ligue1_full_stats_1.csv saved successfully.")


Ligue1_full_stats_1.csv saved successfully.


In [3]:
import pandas as pd
import requests

# Replace with your RapidAPI key
RAPIDAPI_KEY = 'b6e89817d6msh36107de73277139p116779jsne307fb015e33'

# Define the base URL for the fixtures endpoint
url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

# List of seasons to fetch data for
seasons = ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015']  # Season years

# Set the headers with your API key
headers = {
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
    "X-RapidAPI-Key": RAPIDAPI_KEY
}

all_fixtures = []  # To store fixtures for all seasons

# Loop through each season
for season in seasons:
    querystring = {
        "league": "61",  # Ligue 1 league ID
        "season": season
    }

    # Make the API request for the current season
    response = requests.get(url, headers=headers, params=querystring)

    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Assuming the relevant data is in 'response' key
        fixtures = data['response']

        # Add the fixtures to the list
        all_fixtures.extend(fixtures)
    else:
        print(f"Error fetching season {season}: {response.status_code} - {response.text}")

# Convert to DataFrame (flattening any nested data)
df_fixtures = pd.json_normalize(all_fixtures)

# Save to CSV (optional)
df_fixtures.to_csv('Ligue1_full_stats_2.csv', index=False)
print("Data saved to Ligue1_full_stats_2.csv")


Data saved to Ligue1_full_stats_2.csv


In [4]:
df1 = pd.read_csv('Ligue1_full_stats_1.csv')
df2 = pd.read_csv('Ligue1_full_stats_2.csv')

In [11]:
df1.head(2)

,fixture_id,fixture_date,team_name,team_id,Shots on Goal,Shots off Goal,Total Shots,Blocked Shots,Shots insidebox,Shots outsidebox,Fouls,Corner Kicks,Offsides,Ball Possession,Yellow Cards,Red Cards,Goalkeeper Saves,Total passes,Passes accurate,Passes %
0,185545,2015-08-07T18:30:00+00:00,Lille,79,2,4,12,6.0,4,8,0,3,0.0,52%,3.0,0.0,1.0,441,363,82%
1,185545,2015-08-07T18:30:00+00:00,Paris Saint Germain,85,3,2,7,2.0,6,1,0,2,2.0,48%,4.0,1.0,2.0,412,345,84%


In [7]:
# Rename the 'fixture.id' column to 'fixture_id'
df2 = df2.rename(columns={'fixture.id': 'fixture_id'})

In [8]:
final_df = pd.merge(df1, df2, on='fixture_id', how='inner')  # 'inner', 'outer', 'left', 'right'

In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fixture_id              760 non-null    int64  
 1   fixture_date            760 non-null    object 
 2   team_name               760 non-null    object 
 3   team_id                 760 non-null    int64  
 4   Shots on Goal           760 non-null    int64  
 5   Shots off Goal          760 non-null    int64  
 6   Total Shots             760 non-null    int64  
 7   Blocked Shots           748 non-null    float64
 8   Shots insidebox         760 non-null    int64  
 9   Shots outsidebox        760 non-null    int64  
 10  Fouls                   760 non-null    int64  
 11  Corner Kicks            760 non-null    int64  
 12  Offsides                740 non-null    float64
 13  Ball Possession         760 non-null    object 
 14  Yellow Cards            742 non-null    fl

In [10]:
final_df.to_csv('Ligue1_full_stats_last10years.csv', index=False)